## NEW SECCTION ##

In [ ]:
import ezdxf
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, MultiLineString
from shapely.ops import linemerge, unary_union
from scipy.interpolate import splprep, splev
from sklearn.cluster import DBSCAN
import math

# -------------------------
# Parámetros fijos / supuestos
# -------------------------
ASSUME_DXF_UNITS_MM = True   # asumo que las coordenadas DXF están en mm
PASO_MM = 1.0                # paso de muestreo en XY en mm (fijo, denso)
N_Z_STEPS = 50               # pasos en subida/bajada vertical
TRANSIT_ACCEL_FRAC = 0.2     # fracción para rampa en perfil trapezoidal (20% accel, 60% const, 20% decel)
# -------------------------

In [ ]:

# -------------------- 1) Pedir parámetros mínimos (solo 4) --------------------
def pedir_float(msg, default):
    s = input(f"{msg} [{default}]: ").strip()
    return float(s) if s != "" else float(default)

print("UNIDADES: Se asume que el DXF usa mm. Si tu DXF está en m, convierte afuera.")
Z_guardado = pedir_float("Z_guardado [mm]", 200.0)
Z_corte    = pedir_float("Z_corte [mm]", 150.0)
V_trans    = pedir_float("Velocidad_transicion [mm/s]", 50.0)   # para saltos/subidas
V_corte    = pedir_float("Velocidad_corte [mm/s]", 20.0)       # para corte (lineal sobre material)

# convertir velocidades to mm (we keep mm units internally)
paso = PASO_MM
Zg = Z_guardado
Zc = Z_corte
n_z = int(N_Z_STEPS)
